In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn

import pandas as pd
import numpy as np
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

nlp = spacy.load('en')

dtypes ={
    'id': np.uint32,
    'qid1': np.uint32,
    'qid2': np.uint32,
    'question1': np.str,
    'question2': np.str,
    'is_duplicate': np.uint8
}

df = pd.read_csv('../../dataset/quora_train.csv.zip', dtype=dtypes, compression='zip', usecols=['question1', 'question2', 'is_duplicate'], nrows=1000)

In [2]:
def pad(v):
    zeros = np.zeros([100, 300])
    zeros[:v.shape[0],:v.shape[1]] = v
    return zeros
    
q1s = df['question1'].map(nlp).map(lambda x: np.matrix([t.vector for t in x])).map(pad)
q2s = df['question1'].map(nlp).map(lambda x: np.matrix([t.vector for t in x])).map(pad)

q1m = np.concatenate(q1s.values).flatten().reshape([-1, 100, 300])
q2m = np.concatenate(q2s.values).flatten().reshape([-1, 100, 300])

labels = df['is_duplicate'].values

In [3]:
train_q1m, test_q1m = q1m[:700], q1m[700:]
train_q2m, test_q2m = q2m[:700], q2m[700:]
train_labels, test_labels = labels[:700], labels[700:]

In [17]:
tf.reset_default_graph()

Q1 = tf.placeholder(tf.float32, shape=[None, 100, 300])
Q2 = tf.placeholder(tf.float32, shape=[None, 100, 300])
is_dup = tf.placeholder(tf.uint8, shape=[None])
dup_oh = tf.one_hot(is_dup, 2)

cell1 = rnn.BasicLSTMCell(128, forget_bias=0.0)
cell2 = rnn.BasicLSTMCell(128, forget_bias=0.0, reuse=True)

state = cell1.zero_state(100, tf.float32)

#Q1t = tf.transpose(Q1, [1, 0, 2])
#Q2t = tf.transpose(Q2, [1, 0, 2])

outputs1, _1= tf.nn.dynamic_rnn(cell1, Q1, initial_state=state, time_major=True)
outputs2, _2= tf.nn.dynamic_rnn(cell2, Q2, initial_state=state, time_major=True)

output1 = outputs1[:,-1,:]
output2 = outputs2[:,-1,:]

out = tf.concat([output1, output2], axis=1)

In [18]:
keep_prob = tf.placeholder(tf.float32)

W1 = tf.get_variable(shape=[256, 256], initializer=tf.contrib.layers.xavier_initializer(), name='weight1')
b1 = tf.get_variable(shape=[256], initializer=tf.contrib.layers.xavier_initializer(), name='bias1')
y1 = tf.nn.relu(tf.matmul(out, W1) + b1)
y1 = tf.nn.dropout(y1, keep_prob=keep_prob)

W2 = tf.get_variable(shape=[256, 2], initializer=tf.contrib.layers.xavier_initializer(), name='weight2')
b2 = tf.get_variable(shape=[2], initializer=tf.contrib.layers.xavier_initializer(), name='bias2')
y2 = tf.matmul(y1, W2) + b2

In [19]:
cost = tf.nn.softmax_cross_entropy_with_logits(logits=y2, labels=dup_oh)
cost = tf.reduce_mean(cost)

optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [20]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [21]:
from tqdm import tqdm

train_dict = {Q1: train_q1m, Q2: train_q2m, is_dup: train_labels, keep_prob: 0.7}
test_dict = {Q1: test_q1m, Q2: test_q2m, is_dup: test_labels, keep_prob: 1.0}

for i in tqdm(range(10000)):
    sess.run(optimizer, feed_dict=train_dict)
    if i % 10 == 0:
        print(sess.run(cost, feed_dict=train_dict), sess.run(cost, feed_dict=test_dict))


  0%|          | 1/10000 [00:02<6:29:00,  2.33s/it]

0.712927 0.712064



  0%|          | 11/10000 [00:17<4:43:37,  1.70s/it]

0.665561 0.666759


  0%|          | 21/10000 [00:32<4:40:03,  1.68s/it]

...

0.664181 0.6646


  1%|          | 121/10000 [03:05<4:40:44,  1.71s/it]

0.662946 0.664644


  1%|          | 123/10000 [03:08<4:19:06,  1.57s/it]

KeyboardInterrupt: 

In [9]:
variables_names =[v.name for v in tf.trainable_variables()]
values = sess.run(variables_names)
for k,v in zip(variables_names, values):
    print(k, v)

rnn/basic_lstm_cell/weights:0 [[ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 ..., 
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]]
rnn/basic_lstm_cell/biases:0 [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan 

In [9]:
state

LSTMStateTuple(c=<tf.Tensor 'BasicLSTMCellZeroState/zeros:0' shape=(100, 128) dtype=float32>, h=<tf.Tensor 'BasicLSTMCellZeroState/zeros_1:0' shape=(100, 128) dtype=float32>)

In [6]:
outputs1

<tf.Tensor 'rnn/TensorArrayStack/TensorArrayGatherV3:0' shape=(?, 100, 256) dtype=float32>